In [197]:
import numpy as np
import qutip as qp
print("""
Based on the Matheamtica notebook from https://arxiv.org/pdf/1905.06903
""")


Based on the Matheamtica notebook from https://arxiv.org/pdf/1905.06903



In [198]:
def pauli_rot(axis, phi):
    return np.cos(phi) * qp.tensor([one]*len(axis)) + 1j * np.sin(phi) * qp.tensor(axis)

def apply_rot(rho, axis, p1, p2 ,p3):
    rot_0 = pauli_rot(axis, np.pi/8)
    rot_1 = pauli_rot(axis, 5*np.pi/8)
    rot_2 = pauli_rot(axis, -np.pi/8)
    rot_3 = pauli_rot(axis, 3*np.pi/8)

    return (1-p1-p2-p3) * rot_0 * rho*rot_0.dag() + \
                     p1 * rot_1 * rho*rot_1.dag() + \
                     p2 * rot_2 * rho*rot_2.dag() + \
                     p3 * rot_3 * rho*rot_3.dag()

def apply_pauli(rho, pauli, p):
    P = qp.tensor(pauli)
    return (1-p) * rho + p * P * rho * P.dag()

In [199]:
one = qp.identity(2)
x = qp.sigmax()
y = qp.sigmay()
z = qp.sigmaz()
proj_x = (one+x)/2

In [200]:
# magic_state = qp.Qobj(np.array([[1, np.exp(-1j*np.pi/4)]]).conj().T @ np.array([[1, np.exp(-1j*np.pi/4)]])/2)
plus_state = proj_x
magic_state = apply_rot(proj_x, [-z], 0,0,0)


In [201]:
init_4_qubits = qp.tensor(plus_state,plus_state,plus_state,plus_state)
init_5_qubits = qp.tensor(plus_state,plus_state,plus_state,plus_state,plus_state)
init_6_qubits = qp.tensor(plus_state,plus_state,plus_state,plus_state,plus_state,plus_state)


In [202]:
ideal_15_to_1 = qp.tensor(magic_state,plus_state,plus_state,plus_state, plus_state)
ideal_20_to_4 = qp.tensor(magic_state,magic_state,magic_state,magic_state,plus_state,plus_state,plus_state)


In [203]:
pth = 0.01
eth = 0.05
d_scaling = 0.7
def plog(p_phys, e_phys, d, d_scaling=d_scaling):
    assert d > 1
    assert p_phys < pth
    assert e_phys < eth

    x = (p_phys/pth + e_phys/eth)
    if e_phys == 0:
        return 0.1 * x ** ((d+1)/2)
    return 0.1 * x ** (d_scaling * d)


In [204]:
def storage_qubit(rho, single_qubit_error, *ps):
    output = rho.copy()
    for idx, p in enumerate(ps):
        ls = [one]*len(ps)
        ls[idx] = single_qubit_error
        output = apply_pauli(rho ,ls, p)
    return output

def storage_z4_qubit(rho, p1, p2, p3, p4):
    return storage_qubit(rho, z, p1, p2, p3, p4)

def storage_x4_qubit(rho, p1, p2, p3, p4):
    return storage_qubit(rho, x, p1, p2, p3, p4)

def storage_z5_qubit(rho, p1, p2, p3, p4, p5):
    return storage_qubit(rho, z, p1, p2, p3, p4,p5)

def storage_x5_qubit(rho, p1, p2, p3, p4,p5):
    return storage_qubit(rho, x, p1, p2, p3, p4, p5)

def storage_z7_qubit(rho, p1, p2, p3, p4, p5, p6, p7):
    return storage_qubit(rho, z, p1, p2, p3, p4, p5, p6, p7)

def storage_x7_qubit(rho, p1, p2, p3, p4, p5, p6, p7):
    return storage_qubit(rho, x, p1, p2, p3, p4, p5, p6, p7)



In [222]:
def one_level_15_to_1(p_inject, p_phys, e_phys, dx, dz, dm):

    px = plog(p_phys, e_phys, dx)
    pz = plog(p_phys, e_phys, dz)
    pm = plog(p_phys, e_phys, dm)

    output = init_5_qubits.copy()
    p_phys = p_inject

    # Step 1: Apply rotations 1-4
    output = apply_rot(output, [one, z, one, one, one],
                       p_phys/3 + 1/2 * dm/dz * pz * dm,
                       p_phys/3 + 1/2 * dz * pm,
                       p_phys/3)
    output = apply_rot(output, [one, one, z, one, one],
                       p_phys/3 + 1/2 * dm/dz * pz * dm,
                       p_phys/3 + 1/2 * dz * pm,
                       p_phys/3)
    output = apply_rot(output, [one, one, one, z, one],
                       p_phys/3 + 1/2 * dm/dz * pz * dm,
                       p_phys/3 + 1/2 * dz * pm,
                       p_phys/3)
    output = apply_rot(output, [one, z, z, z, one],
                       p_phys/3 + 1/2 * pm * dm,
                       p_phys/3 + 1/2 * pm * dm + 1/2 * (3 * dz * dx)/dm * pm,
                       p_phys/3)

    # Apply storage errors for dm code cycles
    output = storage_x5_qubit(output, 0, 1/2 * dz/dx * px * dm, 1/2 * dz/dx * px * dm,     1/2 * dz/dx * px * dm, 0)
    output = storage_z5_qubit(output, 0, 1/2 * dx/dz * pz * dm, 1/2 * dx/dz * pz * dm,     1/2 * dx/dz * pz * dm, 0)

    # Step 2: Apply rotations 6-7
    output = apply_rot(output, [z, z, z, one, one],
                       p_phys/3 + 1/2 * pm * dm,
                       p_phys/3 + 1/2 * pm * dm + 1/2 * ((dx + 2 * dz) * dx)/dm * pm,
                       p_phys/3)
    output = apply_rot(output, [z, z, one, z, one],
                       p_phys/3 + 1/2 * pm * dm,
                       p_phys/3 + 1/2 * pm * dm + 1/2 * ((dx + 3 * dz) * dx)/dm * pm,
                       p_phys/3)
    output = storage_z5_qubit(output, 1/2 * ((dx + 2 * dz) + (dx + 3 * dz))/dx * px *     dm, 0, 0, 0, 0)

    # Apply storage errors for dm code cycles
    output = storage_x5_qubit(output, 1/2 * px * dm, 1/2 * dz/dx * px * dm, 1/2 * dz/dx     * px * dm, 1/2 * dz/dx * px * dm, 0)
    output = storage_z5_qubit(output, 1/2 * px * dm, 1/2 * dx/dz * pz * dm, 1/2 * dx/dz     * pz * dm, 1/2 * dx/dz * pz * dm, 0)

    # Step 3: Apply rotations 4 and 8-9
    output = apply_rot(output, [z, one, z, z, one],
                       p_phys/3 + 1/2 * pm * dm,
                       p_phys/3 + 1/2 * pm * dm + 1/2 * ((dx + 3 * dz) * dx)/dm * pm,
                       p_phys/3)
    output = apply_rot(output, [z, one, one, z, z],
                       p_phys/3 + 1/2 * pm * dm,
                       p_phys/3 + 1/2 * pm * dm + 1/2 * ((dx + 4 * dz) * dx)/dm * pm,
                       p_phys/3)
    output = apply_rot(output, [one, one, one, one, z],
                       p_phys/3 + 1/2 * dm/dz * pz * dm,
                       p_phys/3 + 1/2 * dz * pm,
                       p_phys/3)
    output = storage_z5_qubit(output, 1/2 * ((dx + 3 * dz) + (dx + 4 * dz))/dx * px *     dm, 0, 0, 0, 0)

    # Apply storage errors for dm code cycles
    output = storage_x5_qubit(output, 1/2 * px * dm, 1/2 * dz/dx * px * dm, 1/2 * dz/dx     * px * dm, 1/2 * dz/dx * px * dm, 1/2 * dz/dx * px * dm)
    output = storage_z5_qubit(output, 1/2 * px * dm, 1/2 * dx/dz * pz * dm, 1/2 * dx/dz     * pz * dm, 1/2 * dx/dz * pz * dm, 1/2 * dx/dz * pz * dm)

    # Step 4: Apply rotations 10-11
    output = apply_rot(output, [z, z, one, one, z],
                       p_phys/3 + 1/2 * pm * dm,
                       p_phys/3 + 1/2 * pm * dm + 1/2 * ((dx + 4 * dz) * dx)/dm * pm,
                       p_phys/3)
    output = apply_rot(output, [z, one, z, one, z],
                       p_phys/3 + 1/2 * pm * dm,
                       p_phys/3 + 1/2 * pm * dm + 1/2 * ((dx + 4 * dz) * dx)/dm * pm,
                       p_phys/3)
    output = storage_z5_qubit(output, 1/2 * ((dx + 4 * dz) + (dx + 4 * dz))/dx * px *     dm, 0, 0, 0, 0)

    # Apply storage errors for dm code cycles
    output = storage_x5_qubit(output, 1/2 * px * dm, 1/2 * dz/dx * px * dm, 1/2 * dz/dx     * px * dm, 1/2 * dz/dx * px * dm, 1/2 * dz/dx * px * dm)
    output = storage_z5_qubit(output, 1/2 * px * dm, 1/2 * dx/dz * pz * dm, 1/2 * dx/dz     * pz * dm, 1/2 * dx/dz * pz * dm, 1/2 * dx/dz * pz * dm)

    # Step 5: Apply rotations 12-13
    output = apply_rot(output, [z, z, z, z, z],
                       p_phys/3 + 1/2 * pm * dm,
                       p_phys/3 + 1/2 * pm * dm + 1/2 * ((dx + 4 * dz) * dx)/dm * pm,
                       p_phys/3)
    output = apply_rot(output, [one, one, z, z, z],
                       p_phys/3 + 1/2 * pm * dm,
                       p_phys/3 + 1/2 * pm * dm + 1/2 * (3 * dz * dx)/dm * pm,
                       p_phys/3)
    output = storage_z5_qubit(output, 1/2 * (dx + 4 * dz)/dx * px * dm, 0, 0, 0, 0)

    # Apply storage errors for dm code cycles
    output = storage_x5_qubit(output, 1/2 * px * (dm + 2 * dx), 1/2 * dz/dx * px * dm,
                              1/2 * dz/dx * px * dm, 1/2 * dz/dx * px * dm, 1/2 * dz/dx     * px * dm)
    output = storage_z5_qubit(output, 1/2 * px * (dm + 2 * dx), 1/2 * dx/dz * pz * dm,
                              1/2 * dx/dz * pz * dm, 1/2 * dx/dz * pz * dm, 1/2 * dx/dz     * pz * dm)

    # Step 6: Apply rotations 14-15
    output = apply_rot(output, [one, z, one, z, z],
                   p_phys/3 + 1/2 * pm * dm,
                   p_phys/3 + 1/2 * pm * dm + 1/2 * (4 * dz * dx)/dm * pm,
                   p_phys/3)
    output = apply_rot(output, [one, z, z, one, z],
                       p_phys/3 + 1/2 * pm * dm,
                       p_phys/3 + 1/2 * pm * dm + 1/2 * (4 * dz * dx)/dm * pm,
                       p_phys/3)

    # Apply storage errors for dm code cycles
    output = storage_x5_qubit(output, 0, 1/2 * dz/dx * px * dm, 1/2 * dz/dx * px * dm,
                              1/2 * dz/dx * px * dm, 1/2 * dz/dx * px * dm)
    output = storage_z5_qubit(output, 0, 1/2 * dx/dz * pz * dm, 1/2 * dx/dz * pz * dm,
                          1/2 * dx/dz * pz * dm, 1/2 * dx/dz * pz * dm)

    return output

In [223]:
def cost_of_one_level_15_to_1(p_inject, p_phys, e_phys, dx, dz, dm):
    out = one_level_15_to_1(p_inject=p_inject,
                            p_phys=p_phys,
                            e_phys=e_phys,
                            dx=dx,
                            dz=dz,
                            dm=dm)

    projection = qp.tensor([one, proj_x,proj_x,proj_x,proj_x])
    p_fail = np.real(1 - (out*projection).tr())
    out_post_sel = 1/(1-p_fail) * projection*out*projection.dag()
    p_out = np.real(1 - (out_post_sel*ideal_15_to_1).tr())
    print(projection.tr(), out.tr(), out_post_sel.tr(), ideal_15_to_1.tr(), (out_post_sel*ideal_15_to_1).tr())
    space_cost = 2*(3*dx*(dx+4*dz)+2*dm)
    time_cost = 6*dm/(1-p_fail)

    print(f"15-to-1, {dx}, {dz}, {dm} with p_phys={p_phys}, e_phys={e_phys}, dx={dx}, dz={dz}, dm={dm}")
    print(f"Output error p_out = {p_out}")
    print(f'Failure Probability p_fail = {p_fail}')
    print(f"Space cost {round(space_cost,2)}")
    print(f"Time code {round(time_cost,2)}")
    print(f"Spacetime cost {round(space_cost*time_cost,2)}")



In [224]:
cost_of_one_level_15_to_1(p_inject=1e-3,
                          p_phys=1e-3,
                          e_phys=0,
                          dx=7,
                          dz=3,
                          dm=3)

2.0 0.9999999999999999 0.9999999999999999 1.0 (0.9999631375729772-5.551115123125783e-17j)
15-to-1, 7, 3, 3 with p_phys=0.001, e_phys=0, dx=7, dz=3, dm=3
Output error p_out = 3.686242702283238e-05
Failure Probability p_fail = 0.14230382668648445
Space cost 810
Time code 20.99
Spacetime cost 16999.03


In [227]:
proj_x

Quantum object: dims=[[2], [2]], shape=(2, 2), type='oper', dtype=CSR, isherm=True
Qobj data =
[[0.5 0.5]
 [0.5 0.5]]

In [228]:
magic_state

Quantum object: dims=[[2], [2]], shape=(2, 2), type='oper', dtype=CSR, isherm=True
Qobj data =
[[0.5       +0.j         0.35355339-0.35355339j]
 [0.35355339+0.35355339j 0.5       +0.j        ]]

In [229]:
one

Quantum object: dims=[[2], [2]], shape=(2, 2), type='oper', dtype=Dia, isherm=True
Qobj data =
[[1. 0.]
 [0. 1.]]